# Importamos todas las librerias necesarias para el proyecto y el database

In [8]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Modelos a comparar
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve,log_loss




#Cargamos el archivo .csv que es el database del proyecto y visualizamos algunas caracteristicas de él

In [11]:
df=pd.read_csv('https://raw.githubusercontent.com/isaavedrab85/PROYECTO/refs/heads/main/PROYECTO/01database.csv',sep=';')


df.shape # tamaño del dataset (filas, columnas)

print(df.head()) # visualización primeras columnas



   RUT SEXO  TIPO COL        COMUNA  P_POND  P_PROM  P_NEM  RK_COL  P_LEN  \
0    1    M         1     PEÑALOLEN  788.70   823.0    771     771    729   
1    2    M         2  SAN BERNARDO  709.25   682.5    785     813    669   
2    3    M         2    LA FLORIDA  683.00   638.0    779     835    607   
3    4    F         3         AYSEN  776.15   680.5    928    1000    656   
4    5    M         2     QUILICURA  858.70   858.0    900     917    799   

   P_M1  ...  C2_24-2  AYG_24-2  C1_24-2  AP_AYG  AP_CI  AP_AL  AP_C2  \
0   917  ...    4.000       0.0      0.0  100000  10000   1000    100   
1   696  ...    5.725       0.0      0.0  100000  10000   1000    100   
2   669  ...    4.800       0.0      0.0  100000  10000   1000    100   
3   705  ...    1.200       0.0      0.0  100000  10000      0      0   
4   917  ...    6.440       0.0      0.0  100000  10000   1000    100   

   AP_AYG_2  AP_CI_2   Total  
0         0        0  111100  
1         0        0  111100  
2    

#Detalles 01DATABASE

Las colulmnas de mi archivo "01database.csv" de base de datos contienen los ID y puntajes variados de pruebas de admisión PAES, diagnóstico y cursos de matemáticas del primer año.

El detalle de cada columna es el siguiente:

### Columna 1: ID de cada estudiante.
### Columna 2: Sexo
*   M = Masculino
*   F = Femenino

### Columna 3: Tipo de Colegio
*   1 = Particular
*   2 = Subencionado
*   3 = Municipal
*   4 = Bicentenerio

### Columna 4: Comuna del colegio.

### Columna 5: Puntaje Ponderado PAES para carrera Ingenieria
$$P_P = \text{NEM} \times 0,1 + P_{Rk}\times 0,3 + P_{CL}\times 0,1+ P_{M_1}\times 0,35 + P_{M_2}\times 0,05 + (P_H\quad o \quad P_C)\times 0,1.$$ Este puntaje va de 100 a 1000 y depende de cada la ponderación que se da a cada variable y que permite ordenar a los postulantes para su posterior matrícula.

### Columna 6: Puntaje Promedio $(P_M)$: $$P_M = \dfrac{P_{M_1} + P_{CL}}{2}.$$ Este puntaje va de 100 a 1000.

Es el promedio de las pruebas de Matemáticas 1 y Comprensión Lectora. Este puntaje va de 100 a 1000.

### Columna 7: Puntaje Notas Enseñanza Media $(NEM)$.

Este es el puntaje que se asigna al promedio de notas obtenido en los 4 años de enseñanza media. Este puntaje va de 100 a 1000.

### Columna 8: Puntaje Ranking $(P_{RK})$.

Este es el puntaje que se asigna a cada estudiante de acuerdo a la posición relativa de sus notas, en la enseñanza media, en comparación con las notas de tres generaciones de alumnos de su mismo colegio. Este puntaje va de 100 a 1000.

### Columna 9: Puntaje Comprensión Lectora $(P_{CL})$.

Es el puntaje obtenido por el estudiante en la PAES de Comprensión Lectora.  Este puntaje va de 100 a 1000.

### Columna 10: Es el puntaje Ponderado sin la PAES M2.

### Columna 11: Puntaje Matemáticas 1 $(P_{M_{1}})$.
Este puntaje va de 100 a 1000.

Es el puntaje obtenido por el estudiante en la PAES de Matemáticas 1. Este puntaje va de 100 a 1000.

### Columna 12: Puntaje Matemáticas 2 $(P_{M_{2}})$.

Es el puntaje obtenido por el estudiante en la PAES de Matemáticas 2. Este puntaje va de 100 a 1000.

### Columna 13: Puntaje Historia $(P_{H})$.

Es el puntaje obtenido por el estudiante en la PAES de Historia. Este puntaje va de 100 a 1000.

### Columna 14: Puntaje Ciencias $(P_{C})$.

Es el puntaje obtenido por el estudiante en la PAES de Ciencias. Este puntaje va de 100 a 1000.

### Columna 15: Máximo puntaje entre $(P_{H})$ y $(P_{C})$.


### Columna 16: Puntaje (%) Diagnóstico $(P_{Diag})$.

Es el porcentaje de logro obtenido en la prueba de Diagnóstico de Matemáticas que se realiza a los estudiantes nuevos. Este puntaje va de 0 a 1.


### Columna 17: Puntaje Álgebra y Geometría 2024-1 $(P_{A\& G})$.

Es la nota obtenida en el curso de Álgebra y Geometría. Esta va de 1 a 7.

### Columna 18: Puntaje Cálculo I 2024-1 $(P_{CI})$.

Es la nota obtenida en el curso de Cálculo I. Esta va de 1 a 7.

### Columna 19: Puntaje Álgebra Lineal 2024-2 $(P_{AL})$.

Es la nota obtenida en el curso de Álgebra Lineal. Este curso se puede rendir sólo si se aprueba el curso de Álgebra y Geometría el primer semestre. Su valor va de 1 a 7.


### Columna 20: Puntaje Cálculo II 2024-2 $(P_{AL})$.

Es la nota obtenida en el curso de Cálculo II. Este curso se puede rendir sólo si se aprueba el curso de Cálculo I el primer semestre. Su valor va de 1 a 7.

### Columna 21: Puntaje Álgebra y Geometría 2024-2 (repetición) $(\widetilde{P_{AL}})$. Es la nota obtenida en el curso de Álgebra y Geometría de repetición. Este curso es rendido por estudiantes que reprobaron en el primer semestre este mismo curso. Su valor va de 1 a 7.



### Columna 22: Puntaje Cálculo I 2024-2 (repetición) $(\widetilde{P_{CI}})$.

Es la nota obtenida en el curso de Cálculo I de repetición. Este curso es rendido por estudiantes que reprobaron en el primer semestre este mismo curso. Su valor va de 1 a 7.

### Columna 23: Etiqueta con un 100000 si el estudiante aprobó el curso AYG_24-1, en caso contrario etiqueta con 0.

### Columna 24: Etiqueta con un 10000 si el estudiante aprobó el curso CI_24-1, en caso contrario etiqueta con 0.

### Columna 25: Etiqueta con un 1000 si el estudiante aprobó el curso AL_24-2, en caso contrario etiqueta con 0.

### Columna 26: Etiqueta con un 100 si el estudiante aprobó el curso CII_24-2, en caso contrario etiqueta con 0.

### Columna 27: Etiqueta con un 10 si el estudiante reprobó AYG_24-1, pero aprobó el curso AYG_24-2, en caso contrario etiqueta con 0.

### Columna 28: Etiqueta con un 1 si el estudiante reprobó CI_24-1, pero aprobó el curso CI_24-2, en caso contrario etiqueta con 0.

### Columna 29: Suma las columnas 23,24,25,26,27 y 28. Esto me permite identificar si un estudiante aprobó o reprobó cuantos y qué cursos.